### Exp 1

Twins-PCPVT-L + QuadTree(B) + pose (deterministic)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../src')

In [3]:
import torch

from torch.utils.data import DataLoader

from data.scannet.utils_scannet import ScanNetDataset
from DEPO.depo import depo_v1
from training.train_depo_pose import train
from training.loss_depo import LossPoseDetermininstic

from transformers import get_scheduler

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Data

In [4]:
train_data = ScanNetDataset(
    root_dir='/home/project/data/scans/',
    npz_path='/home/project/code/data/scannet_splits/smart_sample_train_ft.npz',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz',
    calculate_flow=True
)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True, drop_last=True, pin_memory=True, num_workers=0)

val_data = ScanNetDataset(
    root_dir='/home/project/data/scans/',
    npz_path='/home/project/code/data/scannet_splits/smart_sample_val.npz',
    intrinsics_path='/home/project/ScanNet/scannet_indices/intrinsics.npz',
    calculate_flow=True
)

val_loader = DataLoader(val_data, batch_size=64, shuffle=False, drop_last=False, pin_memory=True, num_workers=0)

#### Config

In [5]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

config = dict(
    experiment_name='DEPO_0',
    device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu'),

    n_epochs=10,
    n_steps_per_epoch=len(train_loader.dataset) // train_loader.batch_size,
    n_accum_steps=1,
    batch_size=train_loader.batch_size,

    swa=False,
    n_epochs_swa=None,
    n_steps_between_swa_updates=None,

    repeat_val_epoch=1,
    repeat_save_epoch=2,

    model_save_path='../../src/weights/DEPO_0'
)

config['n_warmup_steps'] = int(config['n_steps_per_epoch'] * 0.5)

#### Model

In [6]:
model = depo_v0().to(config['device'])

for name, p in model.named_parameters():
    if 'self_encoder' in name:
        p.requires_grad = False
    else:
        p.requires_grad = True

#### Loss & Optimizer & Scheduler

In [7]:
val_loss = LossMixedDetermininstic(mode='val')
train_loss = LossMixedDetermininstic(mode='train')

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-6)

In [9]:
scheduler = get_scheduler(
    "cosine",    
    optimizer=optimizer,
    num_warmup_steps=config['n_warmup_steps'],
    num_training_steps=config['n_steps_per_epoch'] * config['n_epochs']
)

#### Train & val

In [10]:
# data=next(iter(train_loader))
# for key in data.keys():
#     if key in ('image0', 'image1', 'K0', 'K1', 'flow_0to1', 'mask'):
#         data[key] = data[key].to(config['device'])

# B = data['image0'].size(0)
# out = model(
#     img_q=data['image0'], img_s=data['image1'],
#     K_q=data['K0'], K_s=data['K1'],
#     scales_q=0.125 * torch.ones((B, 2), device=config['device']),
#     scales_s=0.125 * torch.ones((B, 2), device=config['device']),
#     H=60, W=80)

In [ ]:
train(model, optimizer, scheduler, train_loss, val_loss, train_loader, val_loader, config, **config)

wandb: Currently logged in as: kovanic. Use `wandb login --relogin` to force relogin


  3%|███▍                                                                                                               | 46/1562 [06:59<3:50:35,  9.13s/it]

### Exp 1

Twins-PCPVT-L + QuadTree(B) + pose (deterministic)

In [ ]:
# model = depo_v1().cuda()

### Exp 2

Twins-PCPVT-L + QuadTree(B) + flow -> pose (deterministic)

In [ ]:
# model = depo_v2().cuda()

In [ ]:
# torch.Tensor([1., 2.], device='cpu')